# Instalação da versão community (**não é a versão educacional**)

- $ python3 -m venv venv #Cria um ambiente virtual
- $ source ./venv/bin/activate #Ativa o ambiente virtual
- $ python3 -m pip install docplex #Instala o docplex
- $ python3 -m pip install cplex #Instala o cplex

# Exemplo 1 - Modelando uma mochila em DocPlex

Seja o seguinte modelo:

$max ~~z = \sum_i (c_i \cdot x_i)$

s.t.

$\sum_i (w_i \cdot x_i) \leq C$

Onde:

* $x_i$ indica se o ítem $i$ é colocado na mochila ou não
* $c_i$ é o ganho monetário por incluir $i$ na mochila
* $w_i$ é o peso do ítem $i$
* $C$ é a capacidade da mochila

In [ ]:
# Dados de um toy model

ci = [5, 10, 1, 20, 10, 10]
wi = [1, 1, 1, 10, 5, 5]

C = 20

In [ ]:
from cplex.callbacks import LazyConstraintCallback
from docplex.mp.callbacks.cb_mixin import *
from docplex.mp.model import Model

In [ ]:
# Criação do modelo

model = Model(name="Mochila", log_output=True)

# Definição de variáveis

# z
z = model.z = model.continuous_var(name="z")

# x
xi = model.xi = model.binary_var_list(len(wi), name="x")


Para a **criação de variáveis**, o Docplex disponibiliza um conjunto de métodos:

- binary_var()	Single binary variable
- binary_var_list()	List of binary variables
- binary_var_dict()	Dictionary of binary variables
- binary_var_matrix()	Matrix of binary variables
- integer_var()	Single integer variable
- integer_var_list()	List of integer variables
- integer_var_dict()	Dictionary of integer variables
- integer_var_matrix()	Matrix of integer variables
- continuous_var()	Single continuous variable
- continuous_var_list()	List of continuous variables
- continuous_var_dict()	Dictionary of continuous variables
- continuous_var_matrix()	Matrix of continuous variables

Links úteis:
* https://ibmdecisionoptimization.github.io/docplex-doc/mp/creating_model.html
* https://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.model.html

In [ ]:
# Definição de restrições

model.maximize(z)

# z = \sum_i (c_i \cdot x_i)
model.add_constraint (z == model.sum(ci[i] * xi[i] for i in range(len(ci))))

# \sum_i (w_i \cdot x_i) \leq C
model.add_constraint (model.sum(wi[i] * xi[i] for i in range(len(ci))) <= C)

**Resolvendo o modelo e obtendo os resultados**

In [ ]:
solucao = model.solve()

In [ ]:
print("z = ",z.solution_value)
for i in range(len(ci)):
    print(f"x{i}={xi[i].solution_value}")

print("Solution status:")
print(solucao.solve_details.status_code, solucao.solve_details.status)

Material relevante:

* Classe **solution**
https://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.solution.html

* Classe **SolveDetails**
https://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.sdetails.html#docplex.mp.sdetails.SolveDetails


# Exemplo 2 - Dimensionamento de lotes

$min~~z = \sum_t (h \cdot I_t + s \cdot y_t)$

$I_0 = 0$

$I_t = I_{(t-1)} + x_t - d_t ~~~ \forall t > 0$

$M \cdot y_t \geq x_t ~~~\forall t$

In [ ]:
# Dados do toy

di = [0, 100, 0, 200, 50, 100, 0, 100]
h = 1
s = 300

In [ ]:
# Criação do modelo

model = Model(name="LotSizing", log_output=True)

# Definição de variáveis

z = model.z = model.continuous_var(name="z")

xt = model.xt = model.continuous_var_list(len(di), name="x")

yt = model.yt = model.binary_var_list(len(di), name="y")

It = model.it = model.continuous_var_list(len(di), name="i")

# Definição de restrições

# Definição de restrições

model.minimize(z)

# z = \sum_i (c_i \cdot x_i)
model.add_constraint (z == model.sum(
    h * It[t] + s * yt[t]
    for t in range(len(di))
))

# I_t = I_{(t-1)} + x_t - d_t   \forall t > 0
model.add_constraints (
    It[t] == It[t-1] + xt[t] - di[t]
    for t in range(1, len(di))
)

#M \cdot y_t \geq x_t ~~~\forall t
model.add_constraints (
    100000 * yt[t] >= xt[t]
    for t in range(1, len(di))
)

#I0 = 0
It[0].set_ub(0)

In [ ]:
solucao = model.solve()

In [ ]:
#Resultados:

print(f"FO: {z.solution_value}")
print("i\tIt\txt\tyt")
for t in range(len(di)):
    print(f"{t}\t{It[t].solution_value}\t{xt[t].solution_value}\t{yt[t].solution_value}")

## Salvando modelo e solução

In [ ]:
model.export_as_lp("losizing.lp")

solucao.export_as_sol(".", "lotsizing.sol")

# Implementando callbacks

Vamos supor o modelo anterior:

$min~~z = \sum_t (h \cdot I_t + s \cdot y_t)$

$I_0 = 0$

$I_t = I_{(t-1)} + x_t - d_t$

$M \cdot y_t \geq x_t ~~~\forall t$

Vamos adicionar uma nova restrição:

$x_t \leq 300 ~~~\forall t$

Mas, agora, queremos adicionar essa restrição apenas se ela for violada. Para isso, vamos usar um callback:

(obs: mais exemplos em: https://github.com/IBMDecisionOptimization/docplex-examples/tree/master/examples/mp/callbacks )

In [ ]:
di = [0, 100, 0, 200, 50, 100, 0, 100]
h = 1
s = 300

# Criação do modelo

model = Model(name="LotSizing", log_output=True)

# Definição de variáveis

z = model.z = model.continuous_var(name="z")

xt = model.xt = model.continuous_var_list(len(di), name="x")

yt = model.yt = model.binary_var_list(len(di), name="y")

It = model.it = model.continuous_var_list(len(di), name="i")

# Definição de restrições

# Definição de restrições

model.minimize(z)

# z = \sum_i (c_i \cdot x_i)
model.add_constraint (z == model.sum(
    h * It[t] + s * yt[t]
    for t in range(len(di))
))

# I_t = I_{(t-1)} + x_t - d_t   \forall t > 0
model.add_constraints (
    It[t] == It[t-1] + xt[t] - di[t]
    for t in range(1, len(di))
)

#M \cdot y_t \geq x_t ~~~\forall t
model.add_constraints (
    100000 * yt[t] >= xt[t]
    for t in range(1, len(di))
)

#I0 = 0
It[0].set_ub(0)

In [ ]:
class MeuCallBack(ConstraintCallbackMixin, LazyConstraintCallback):
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)
    
    @print_called('--> lazy constraint callback called: #{0}')
    def __call__(self):

        xt = self.make_solution_from_vars(self.model.xt)
        for t in range(len(di)):
            if xt.get_value(self.model.xt[t]) > 300:
                sol = self.make_complete_solution()
                cts = [self.model.xt[t] <= 300]
                unsats = self.get_cpx_unsatisfied_cts(cts, sol, tolerance=1e-6)
                for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)

                print("Corte adicionado!")
                print(cts)
                break


meu_callback = model.register_callback(MeuCallBack)

solucao = model.solve()



print(f"FO: {z.solution_value}")
print("i\tIt\txt\tyt")
for t in range(len(di)):
    print(f"{t}\t{It[t].solution_value}\t{xt[t].solution_value}\t{yt[t].solution_value}")